In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [2]:
%run ./model_end_events_for_outages_METHODS.ipynb

In [3]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
import string
from pathlib import Path
import json
import pickle
import joblib

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy
from functools import reduce

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from MECPOCollection import MECPOCollection
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutageModeler import OutageModeler
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from CustomJSON import CustomEncoder, CustomWriter

In [4]:
from sklearn.preprocessing import OrdinalEncoder

In [5]:
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras

import scipy

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# ---------------------------------------------------------------------------------------------------

In [6]:
fig_num=0
data_dir_base = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data'

# save_dir_model_base = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20230615\Models_00_05'
# save_dir_model_base = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20230615\Models'
save_dir_model_base = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231221\Models'


In [7]:
save_results = False
save_model   = False

# save_dir_model = None
save_dir_model = 'All_EEMSP_agg_Top10_v3'
if save_dir_model is None:
    save_dir_model = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir_model = os.path.join(save_dir_model_base, save_dir_model)
#-----
if not os.path.exists(save_dir_model) and (save_results or save_model):
    os.makedirs(save_dir_model)

In [8]:
build_mecpo_colls = True

In [9]:
if build_mecpo_colls:
    verbose = True
    #----- Outages -------------------------------------------------------------
    # run_date_outg = '20230615'
    # event_date_ranges_outg = [
    #     ['2020-01-01', '2020-12-31'],
    #     ['2021-01-01', '2021-12-31'], 
    #     ['2022-01-01', '2022-09-30'], 
    # ]
    # run_date_outg = '20231201'
    # event_date_ranges_outg = [
    #     ['2023-04-01', '2023-09-30'], 
    # ]
    run_date_outg = '20231221'
    event_date_ranges_outg = [
        ['2023-04-01', '2023-11-30'], 
    ]
    grp_by_cols_outg = ['outg_rec_nb', 'trsf_pole_nb']    
    #----- No Outages ----------------------------------------------------------
    # run_date_otbl = '20230512'
    # event_date_ranges_otbl = [
    #     ['2022-01-01', '2022-12-31'], 
    # ]
    # run_date_otbl = '20231201'
    # event_date_ranges_otbl = [
    #     ['2023-04-01', '2023-09-30'], 
    # ]
    run_date_otbl = '20231221'
    event_date_ranges_otbl = [
        ['2023-04-01', '2023-11-30'], 
    ]
    grp_by_cols_otbl = ['trsf_pole_nb', 'no_outg_rec_nb']    
    #----- No Outages Pristine -------------------------------------------------
    # run_date_prbl = '20230301'
    # event_date_ranges_prbl = [
    #     ['2022-01-01', '2022-12-31'], 
    # ]
    # run_date_prbl = '20231201'
    # event_date_ranges_prbl = [
    #     ['2023-04-01', '2023-09-30'], 
    # ]
    run_date_prbl = '20231221'
    event_date_ranges_prbl = [
        ['2023-04-01', '2023-11-30'], 
    ]
    grp_by_cols_prbl = ['trsf_pole_nb', 'no_outg_rec_nb']    
    #---------------------------------------------------------------------------
    normalize_by_time_interval  = True
    #-----
    include_power_down_minus_up = False
    pd_col = 'Primary Power Down'
    pu_col = 'Primary Power Up'
    pd_m_pu_col = 'Power Down Minus Up'
    #-----
    regex_to_remove_patterns    = ['.*cleared.*', '.*Test Mode.*']
    regex_to_remove_ignore_case = True
    #---------------------------------------------------------------------------
    max_total_counts = None
    # max_total_counts=150
    # max_total_counts={
    #     '01-05 Days':150, 
    #     '06-10 Days':150, 
    #     '11-15 Days':150, 
    #     '16-20 Days':150,
    #     '21-25 Days':150, 
    #     '26-30 Days':150
    # }
    how_max_total_counts='any'
    #---------------------------------------------------------------------------    
    rcpo_dfs_name_outg = 'rcpo_df_norm_by_xfmr_nSNs'
    rcpo_dfs_name_otbl = 'rcpo_df_norm_by_xfmr_nSNs'
    rcpo_dfs_name_prbl = 'rcpo_df_norm_by_xfmr_nSNs'

    mecpo_idx_for_ordering = 0
    #-------------------------
    icpo_dfs_name_outg          = 'i'+rcpo_dfs_name_outg[1:]
    icpo_dfs_name_otbl       = 'i'+rcpo_dfs_name_otbl[1:]
    icpo_dfs_name_prbl = 'i'+rcpo_dfs_name_prbl[1:]    
    #---------------------------------------------------------------------------
    freq='5D'
    days_min_max_outg_td_windows=[
        [1,6], [6,11], [11,16], [16,21], [21,26], [26,31]
    ]
    old_to_new_keys_dict = {
        'outg_td_window_1_to_6_days'  :'01-06 Days',
        'outg_td_window_6_to_11_days' :'06-11 Days',
        'outg_td_window_11_to_16_days':'11-16 Days',
        'outg_td_window_16_to_21_days':'16-21 Days',
        'outg_td_window_21_to_26_days':'21-26 Days',
        'outg_td_window_26_to_31_days':'26-31 Days'
    }

    #-------------------------
    assert(len(old_to_new_keys_dict)==len(days_min_max_outg_td_windows))
    #-------------------------
    # Sanity check
    for window_i in days_min_max_outg_td_windows:
        assert(pd.Timedelta(f'{window_i[1]}D')-pd.Timedelta(f'{window_i[0]}D')==pd.Timedelta(freq))

In [10]:
grp_by_cols_otbl

['trsf_pole_nb', 'no_outg_rec_nb']

In [11]:
days_min_max_outg_td_windows

[[1, 6], [6, 11], [11, 16], [16, 21], [21, 26], [26, 31]]

In [12]:
old_to_new_keys_dict

{'outg_td_window_1_to_6_days': '01-06 Days',
 'outg_td_window_6_to_11_days': '06-11 Days',
 'outg_td_window_11_to_16_days': '11-16 Days',
 'outg_td_window_16_to_21_days': '16-21 Days',
 'outg_td_window_21_to_26_days': '21-26 Days',
 'outg_td_window_26_to_31_days': '26-31 Days'}

In [13]:
base_dirs_outg = get_mecpx_base_dirs(
    dataset          = 'outg', 
    acq_run_date     = run_date_outg, 
    data_date_ranges = event_date_ranges_outg, 
    data_dir_base    = data_dir_base, 
    assert_found     = True
)

base_dirs_otbl = get_mecpx_base_dirs(
    dataset          = 'otbl', 
    acq_run_date     = run_date_otbl, 
    data_date_ranges = event_date_ranges_otbl, 
    data_dir_base    = data_dir_base, 
    assert_found     = True
)

base_dirs_prbl = get_mecpx_base_dirs(
    dataset          = 'prbl', 
    acq_run_date     = run_date_prbl, 
    data_date_ranges = event_date_ranges_prbl, 
    data_dir_base    = data_dir_base, 
    assert_found     = True
)

In [14]:
# NOT REALLY NEEDED!
# ede_data_dirs_ will be needed later for building time_infos_dfs, but these locations
#   will be stored in the OutageModeler object
if build_mecpo_colls:
    #----------------------------------------------------------------------------------------------------
    pkl_dirs_dict_outg = get_rcpo_pkl_dirs_for_build_and_combine_mecpo_colls_for_dates(
        dataset          = 'outg', 
        acq_run_date     = run_date_outg, 
        data_date_ranges = event_date_ranges_outg, 
        grp_by_cols      = grp_by_cols_outg, 
        data_dir_base    = data_dir_base
    )
    #-----
    pkl_dirs_dict_otbl = get_rcpo_pkl_dirs_for_build_and_combine_mecpo_colls_for_dates(
        dataset          = 'otbl', 
        acq_run_date     = run_date_otbl, 
        data_date_ranges = event_date_ranges_otbl, 
        grp_by_cols      = grp_by_cols_otbl, 
        data_dir_base    = data_dir_base
    )
    #-----
    pkl_dirs_dict_prbl = get_rcpo_pkl_dirs_for_build_and_combine_mecpo_colls_for_dates(
        dataset          = 'prbl', 
        acq_run_date     = run_date_prbl, 
        data_date_ranges = event_date_ranges_prbl, 
        grp_by_cols      = grp_by_cols_prbl, 
        data_dir_base    = data_dir_base
    )
    #----------------------------------------------------------------------------------------------------
    ede_data_dirs_outg = [os.path.join(Path(x).parent, 'EndEvents') for x in pkl_dirs_dict_outg.values()]
    ede_data_dirs_otbl = [os.path.join(Path(x).parent, 'EndEvents') for x in pkl_dirs_dict_otbl.values()]
    ede_data_dirs_prbl = [os.path.join(Path(x).parent, 'EndEvents') for x in pkl_dirs_dict_prbl.values()]
    #-----
    assert(all([os.path.exists(x) for x in ede_data_dirs_outg]))
    assert(all([os.path.exists(x) for x in ede_data_dirs_otbl]))
    assert(all([os.path.exists(x) for x in ede_data_dirs_prbl]))

In [15]:
if build_mecpo_colls:
    mecpo_build_info_dict = dict()
    #-------------------------
    mecpo_build_info_dict['run_date_outg']                   = run_date_outg
    mecpo_build_info_dict['event_date_ranges_outg']          = event_date_ranges_outg
    mecpo_build_info_dict['grp_by_cols_outg']                = grp_by_cols_outg
    #-----
    mecpo_build_info_dict['run_date_otbl']                   = run_date_otbl
    mecpo_build_info_dict['event_date_ranges_otbl']          = event_date_ranges_otbl
    mecpo_build_info_dict['grp_by_cols_otbl']                = grp_by_cols_otbl
    #-----
    mecpo_build_info_dict['run_date_prbl']                   = run_date_prbl
    mecpo_build_info_dict['event_date_ranges_prbl']          = event_date_ranges_prbl
    mecpo_build_info_dict['grp_by_cols_prbl']                = grp_by_cols_prbl
    #-------------------------
    mecpo_build_info_dict['normalize_by_time_interval']      = normalize_by_time_interval
    #-------------------------
    mecpo_build_info_dict['include_power_down_minus_up']     = include_power_down_minus_up
    mecpo_build_info_dict['pd_col']                          = pd_col
    mecpo_build_info_dict['pu_col']                          = pu_col
    mecpo_build_info_dict['pd_m_pu_col']                     = pd_m_pu_col
    #-------------------------
    mecpo_build_info_dict['regex_to_remove_patterns']        = regex_to_remove_patterns
    mecpo_build_info_dict['regex_to_remove_ignore_case']     = regex_to_remove_ignore_case
    #-------------------------
    mecpo_build_info_dict['max_total_counts']                = max_total_counts
    mecpo_build_info_dict['how_max_total_counts']            = how_max_total_counts
    #-------------------------
    mecpo_build_info_dict['mecpo_idx_for_ordering']          = mecpo_idx_for_ordering
    #-------------------------
    mecpo_build_info_dict['rcpo_dfs_name_outg']              = rcpo_dfs_name_outg
    mecpo_build_info_dict['rcpo_dfs_name_otbl']              = rcpo_dfs_name_otbl
    mecpo_build_info_dict['rcpo_dfs_name_prbl']              = rcpo_dfs_name_prbl
    #-----
    mecpo_build_info_dict['icpo_dfs_name_outg']              = icpo_dfs_name_outg
    mecpo_build_info_dict['icpo_dfs_name_otbl']              = icpo_dfs_name_otbl
    mecpo_build_info_dict['icpo_dfs_name_prbl']              = icpo_dfs_name_prbl
    #-------------------------
    mecpo_build_info_dict['freq']                            = freq
    #-------------------------
    mecpo_build_info_dict['days_min_max_outg_td_windows']    = days_min_max_outg_td_windows
    #-------------------------
    mecpo_build_info_dict['old_to_new_keys_dict']            = old_to_new_keys_dict
    #-------------------------
    mecpo_build_info_dict['pkl_dirs_dict_outg']              = pkl_dirs_dict_outg
    mecpo_build_info_dict['pkl_dirs_dict_otbl']              = pkl_dirs_dict_otbl
    mecpo_build_info_dict['pkl_dirs_dict_prbl']              = pkl_dirs_dict_prbl
    #-----
    mecpo_build_info_dict['ede_data_dirs_outg']              = ede_data_dirs_outg
    mecpo_build_info_dict['ede_data_dirs_otbl']              = ede_data_dirs_otbl
    mecpo_build_info_dict['ede_data_dirs_prbl']              = ede_data_dirs_prbl

# Build MECPOCollection objects

In [16]:
if build_mecpo_colls:
    #----------------------------------------------------------------------------------------------------
    # Build MECPOCollection objects
    #----------------------------------------------------------------------------------------------------
    #----- Outages ------------------------------------
    mecpo_coll_outg = build_and_combine_mecpo_colls_for_dates(
        dataset                      = 'outg', 
        acq_run_date                 = run_date_outg, 
        data_date_ranges             = event_date_ranges_outg, 
        grp_by_cols                  = grp_by_cols_outg, 
        days_min_max_outg_td_windows = days_min_max_outg_td_windows, 
        old_to_new_keys_dict         = old_to_new_keys_dict, 
        coll_label                   = 'Outages (All Xfmrs)', 
        barplot_kwargs_shared        = dict(facecolor='red'), 
        normalize_by_time_interval   = normalize_by_time_interval, 
        data_dir_base                = data_dir_base
    )
    
    #----- No Outages ---------------------------------
    mecpo_coll_otbl = build_and_combine_mecpo_colls_for_dates(
        dataset                      = 'otbl', 
        acq_run_date                 = run_date_otbl, 
        data_date_ranges             = event_date_ranges_otbl, 
        grp_by_cols                  = grp_by_cols_otbl, 
        days_min_max_outg_td_windows = days_min_max_outg_td_windows, 
        old_to_new_keys_dict         = old_to_new_keys_dict, 
        coll_label                   = 'No Outages', 
        barplot_kwargs_shared        = dict(facecolor='orange'), 
        normalize_by_time_interval   = normalize_by_time_interval, 
        data_dir_base                = data_dir_base
    )    
    
    #----- No Outages Pristine ------------------------
    mecpo_coll_prbl = build_and_combine_mecpo_colls_for_dates(
        dataset                      = 'prbl', 
        acq_run_date                 = run_date_prbl, 
        data_date_ranges             = event_date_ranges_prbl, 
        grp_by_cols                  = grp_by_cols_prbl, 
        days_min_max_outg_td_windows = days_min_max_outg_td_windows, 
        old_to_new_keys_dict         = old_to_new_keys_dict, 
        coll_label                   = 'No Outages', 
        barplot_kwargs_shared        = dict(facecolor='orange'), 
        normalize_by_time_interval   = normalize_by_time_interval, 
        data_dir_base                = data_dir_base
    )    

In [17]:
# def build_mecpx_colls_for_modeler(
#     acq_run_date, 
#     data_date_ranges, 
#     data_dir_base, 
#     days_min_max_outg_td_windows, 
#     old_to_new_keys_dict, 
#     grp_by_cols_outg = ['outg_rec_nb', 'trsf_pole_nb'], 
#     grp_by_cols_bsln = ['trsf_pole_nb', 'no_outg_rec_nb'], 
#     include_prbl=True, 
#     normalize_by_time_interval=True, 
#     **kwargs
# ):
#     r"""
#     Build the MECPOCollection objects for outage data (outg), outage transformers baseline data (otbl), and pristine baseline (prbl)
#       to be used by the modeler.
#     This function implicitly assumes outg, otbl, and prbl all share the same acq_run_date, data_date_ranges, and data_dir_base (and 
#       that otbl and prbl share grp_by_cols_bsln).
#     If outg, otbl, and/or prbl do not share the same acq_run_date, data_date_ranges, and/or data_dir_base, then the dataset specific 
#       version of those parameters should be used (e.g., acq_run_date_outg, acq_run_date_otbl, acq_run_date_prbl)

#     kwargs:
#         As stated above, these are mainly used when outg, otbl, and/or prbl do not share the same input parameters.
#         There are also kwargs hidden for setting coll_label, barplot_kwargs_shared
#         Acceptable kwargs:
#             Any of the following are acceptable with _outg, _otbl, or _prbl tags
#             'acq_run_date',        (e.g., acq_run_date_outg, acq_run_date_otbl, acq_run_date_prbl)
#             'data_date_ranges', 
#             'data_dir_base', 
#             'grp_by_cols', 
#             'coll_label', 
#             'barplot_kwargs_shared'
#     """
#     #----------------------------------------------------------------------------------------------------
#     acceptable_kwargs = ['acq_run_date', 'data_date_ranges', 'data_dir_base', 'coll_label', 'barplot_kwargs_shared']
#     acceptable_kwargs = list(itertools.chain.from_iterable([[f'{x}_outg', f'{x}_otbl', f'{x}_prbl'] for x in acceptable_kwargs]))
#     acceptable_kwargs.extend(['grp_by_cols_otbl', 'grp_by_cols_prbl'])
#     assert(set(kwargs.keys()).difference(set(acceptable_kwargs))==set())
    
#     #----- Outages (_outg) ------------------------------------------------------------------------------
#     acq_run_date_outg          = kwargs.get('acq_run_date_outg',          acq_run_date)
#     data_date_ranges_outg      = kwargs.get('data_date_ranges_outg',      data_date_ranges)
#     data_dir_base_outg         = kwargs.get('data_dir_base_outg',         data_dir_base)
#     # grp_by_cols_outg           = kwargs.get('grp_by_cols_outg',           grp_by_cols_outg)
#     coll_label_outg            = kwargs.get('coll_label_outg',            'Outages')
#     barplot_kwargs_shared_outg = kwargs.get('barplot_kwargs_shared_outg', dict(facecolor='red'))
#     #-------------------------
#     mecpx_coll_outg = build_and_combine_mecpo_colls_for_dates(
#         dataset                      = 'outg', 
#         acq_run_date                 = acq_run_date_outg, 
#         data_date_ranges             = data_date_ranges_outg, 
#         grp_by_cols                  = grp_by_cols_outg, 
#         days_min_max_outg_td_windows = days_min_max_outg_td_windows, 
#         old_to_new_keys_dict         = old_to_new_keys_dict, 
#         coll_label                   = coll_label_outg, 
#         barplot_kwargs_shared        = barplot_kwargs_shared_outg, 
#         normalize_by_time_interval   = normalize_by_time_interval, 
#         data_dir_base                = data_dir_base_outg
#     )

#     #----- Outage Transformers Baseline (_otbl) ---------------------------------------------------------
#     acq_run_date_otbl          = kwargs.get('acq_run_date_otbl',          acq_run_date)
#     data_date_ranges_otbl      = kwargs.get('data_date_ranges_otbl',      data_date_ranges)
#     data_dir_base_otbl         = kwargs.get('data_dir_base_otbl',         data_dir_base)
#     grp_by_cols_otbl           = kwargs.get('grp_by_cols_otbl',           grp_by_cols_bsln)
#     coll_label_otbl            = kwargs.get('coll_label_otbl',            'Baseline (OTBL)')
#     barplot_kwargs_shared_otbl = kwargs.get('barplot_kwargs_shared_otbl', dict(facecolor='orange'))
#     #-------------------------
#     mecpx_coll_otbl = build_and_combine_mecpo_colls_for_dates(
#         dataset                      = 'otbl', 
#         acq_run_date                 = acq_run_date_otbl, 
#         data_date_ranges             = data_date_ranges_otbl, 
#         grp_by_cols                  = grp_by_cols_otbl, 
#         days_min_max_outg_td_windows = days_min_max_outg_td_windows, 
#         old_to_new_keys_dict         = old_to_new_keys_dict, 
#         coll_label                   = coll_label_otbl, 
#         barplot_kwargs_shared        = barplot_kwargs_shared_otbl, 
#         normalize_by_time_interval   = normalize_by_time_interval, 
#         data_dir_base                = data_dir_base_otbl
#     )

#     #----- Pristine Baseline (_prbl) --------------------------------------------------------------------
#     if include_prbl:
#         acq_run_date_prbl          = kwargs.get('acq_run_date_prbl',          acq_run_date)
#         data_date_ranges_prbl      = kwargs.get('data_date_ranges_prbl',      data_date_ranges)
#         data_dir_base_prbl         = kwargs.get('data_dir_base_prbl',         data_dir_base)
#         grp_by_cols_prbl           = kwargs.get('grp_by_cols_prbl',           grp_by_cols_bsln)
#         coll_label_prbl            = kwargs.get('coll_label_prbl',            'Baseline (PRBL)')
#         barplot_kwargs_shared_prbl = kwargs.get('barplot_kwargs_shared_prbl', dict(facecolor='orange'))
#         #-------------------------
#         mecpx_coll_prbl = build_and_combine_mecpo_colls_for_dates(
#             dataset                      = 'prbl', 
#             acq_run_date                 = acq_run_date_prbl, 
#             data_date_ranges             = data_date_ranges_prbl, 
#             grp_by_cols                  = grp_by_cols_prbl, 
#             days_min_max_outg_td_windows = days_min_max_outg_td_windows, 
#             old_to_new_keys_dict         = old_to_new_keys_dict, 
#             coll_label                   = coll_label_prbl, 
#             barplot_kwargs_shared        = barplot_kwargs_shared_prbl, 
#             normalize_by_time_interval   = normalize_by_time_interval, 
#             data_dir_base                = data_dir_base_prbl
#         )
#     else:
#         mecpx_coll_prbl = None

#     #----------------------------------------------------------------------------------------------------
#     return_dict = dict(
#         outg = mecpx_coll_outg, 
#         otbl = mecpx_coll_otbl, 
#         prbl = mecpx_coll_prbl
#     )
#     return return_dict

In [18]:
we_dict = OutageModeler.build_mecpx_colls_for_modeler(
    acq_run_date                 = run_date_outg, 
    data_date_ranges             = event_date_ranges_outg, 
    data_dir_base                = data_dir_base, 
    days_min_max_outg_td_windows = days_min_max_outg_td_windows, 
    old_to_new_keys_dict         = old_to_new_keys_dict, 
    grp_by_cols_outg             = grp_by_cols_outg,   
    grp_by_cols_bsln             = grp_by_cols_otbl, 
    include_prbl                 = True, 
    normalize_by_time_interval   = normalize_by_time_interval
)

In [19]:
# def set_mecpx_colls(
#     self, 
#     mecpx_coll_outg, 
#     mecpx_coll_otbl, 
#     mecpx_coll_prbl = None
# ):
#     r"""
#     """
#     #-------------------------
#     assert(isinstance(mecpx_coll_outg, MECPOCollection))
#     assert(isinstance(mecpx_coll_otbl, MECPOCollection))
#     if mecpx_coll_prbl is not None:
#         assert(isinstance(mecpx_coll_prbl, MECPOCollection))
#     #-------------------------
#     self.mecpx_coll_outg = mecpx_coll_outg
#     self.mecpx_coll_otbl = mecpx_coll_otbl
#     self.mecpx_coll_prbl = mecpx_coll_prbl

In [20]:
# def perform_similarity_operations(
#     self, 
#     verbose=True
# ):
#     r"""
#     Make the columns equal in the relevant DFs from the MECPOCollections
#     """
#     #----------------------------------------------------------------------------------------------------
#     assert(self.mecpx_coll_outg is not None and self.mecpx_coll_otbl is not None)
#     mecpo_colls_with_cpo_df_names = [
#         [self.mecpx_coll_outg, self.cpx_dfs_name_outg], 
#         [self.mecpx_coll_otbl, self.cpx_dfs_name_otbl]
#     ]
#     #-----
#     if self.mecpx_coll_prbl is not None:
#         mecpo_colls_with_cpo_df_names.append([self.mecpx_coll_prbl, self.cpx_dfs_name_prbl])
#     #----------------------------------------------------------------------------------------------------
#     if verbose:
#         print('In OutageModeler.perform_similarity_operations\nStarting shapes:')
#         for i_coll, mecpo_coll_i_w_df_name in enumerate(mecpo_colls_with_cpo_df_names):
#             print(f"MECPO Collection {i_coll}")
#             for an_key in mecpo_coll_i_w_df_name[0].mecpo_an_keys:
#                 print(f"\t\t{an_key}, '{rcpo_dfs_name_outg}' shape: {mecpo_coll_i_w_df_name[0].get_cpo_df(an_key, rcpo_dfs_name_outg).shape}")
#     #----------------------------------------------------------------------------------------------------
#     # First, make columns equal between MECPOAn objects within each MECPOCollection
#     for mecpo_coll_i_w_df_name in mecpo_colls_with_cpo_df_names:
#         mecpo_coll_i_w_df_name[0].make_cpo_columns_equal(drop_empty_cpo_dfs=True)

#     #-------------------------
#     # Now, make columns equal between the MECPOCollections
#     MECPOCollection.make_cpo_columns_equal_between_mecpo_colls(
#         mecpo_colls = [x[0] for x in mecpo_colls_with_cpo_df_names], 
#         drop_empty_cpo_dfs=True
#     )

#     #-------------------------
#     # If not all same cpo_df names are used between collections, then one should call 
#     #   MECPOCollection.make_mixed_cpo_columns_equal_between_mecpo_colls.
#     if len(set([x[1] for x in mecpo_colls_with_cpo_df_names]))>1:
#         MECPOCollection.make_mixed_cpo_columns_equal_between_mecpo_colls(
#             mecpo_colls_with_cpo_df_names = mecpo_colls_with_cpo_df_names, 
#             segregate_by_mecpo_an_keys=False
#         )
        
#     #----------------------------------------------------------------------------------------------------
#     if verbose:
#         print('\n\nAfter making columns equal amongst collections:')
#         for i_coll, mecpo_coll_i_w_df_name in enumerate(mecpo_colls_with_cpo_df_names):
#             print(f"MECPO Collection {i_coll}")
#             for an_key in mecpo_coll_i_w_df_name[0].mecpo_an_keys:
#                 print(f"\t\t{an_key}, '{rcpo_dfs_name_outg}' shape: {mecpo_coll_i_w_df_name[0].get_cpo_df(an_key, rcpo_dfs_name_outg).shape}")

In [21]:
# # NOTE: This is resource wasting as it performs reductions on all DFs, not just those of the appropriate name
# def perform_reduction_operations(
#     self, 
#     verbose=True
# ):
#     r"""
#     Remove and/or combine reasons
#     Also, insert power_down_minus_up column if set.
#     """
#     #----------------------------------------------------------------------------------------------------
#     assert(self.mecpx_coll_outg is not None and self.mecpx_coll_otbl is not None)
#     mecpo_colls_with_cpo_df_names = [
#         [self.mecpx_coll_outg, self.cpx_dfs_name_outg], 
#         [self.mecpx_coll_otbl, self.cpx_dfs_name_otbl]
#     ]
#     #-----
#     if self.mecpx_coll_prbl is not None:
#         mecpo_colls_with_cpo_df_names.append([self.mecpx_coll_prbl, self.cpx_dfs_name_prbl])
#     #----------------------------------------------------------------------------------------------------
#     if verbose:
#         print('In OutageModeler.perform_reduction_operations\nStarting shapes:')
#         for i_coll, mecpo_coll_i_w_df_name in enumerate(mecpo_colls_with_cpo_df_names):
#             print(f"MECPO Collection {i_coll}")
#             for an_key in mecpo_coll_i_w_df_name[0].mecpo_an_keys:
#                 print(f"\t\t{an_key}, '{rcpo_dfs_name_outg}' shape: {mecpo_coll_i_w_df_name[0].get_cpo_df(an_key, rcpo_dfs_name_outg).shape}")

#     #----------------------------------------------------------------------------------------------------
#     # Remove and/or combine reasons
#     #----------------------------------------------------------------------------------------------------
#     #-------------------------
#     # Remove all reasons containing 'cleared'
#     for mecpo_coll_i_w_df_name in mecpo_colls_with_cpo_df_names:
#         mecpo_coll_i_w_df_name[0].remove_reasons_from_all_rcpo_dfs(
#             regex_patterns_to_remove = self.mecpx_build_info_dict['regex_to_remove_patterns'], 
#             ignore_case              = self.mecpx_build_info_dict['regex_to_remove_ignore_case']
#         )

#     #-------------------------
#     # Combine reasons using the standard combine (see dflt_patterns_and_replace in MECPODf.combine_cpo_df_reasons
#     #   for the list of default patterns_and_replace)
#     for mecpo_coll_i_w_df_name in mecpo_colls_with_cpo_df_names:
#         mecpo_coll_i_w_df_name[0].combine_reasons_in_all_rcpo_dfs(**self.combine_reasons_kwargs)

#     #-------------------------
#     # Build power down minus power up counts
#     if self.mecpx_build_info_dict['include_power_down_minus_up']:
#         for mecpo_coll_i_w_df_name in mecpo_colls_with_cpo_df_names:
#             mecpo_coll_i_w_df_name[0].delta_cpo_df_reasons_in_all_rcpo_dfs(
#                 reasons_1         = self.mecpx_build_info_dict['pd_col'],
#                 reasons_2         = self.mecpx_build_info_dict['pu_col'],
#                 delta_reason_name = self.mecpx_build_info_dict['pd_m_pu_col']
#             )        
#     #-------------------------
#     # Don't want to include SNs or nSNs cols (and similar) in plotting, so remove
#     for mecpo_coll_i_w_df_name in mecpo_colls_with_cpo_df_names:
#         mecpo_coll_i_w_df_name[0].remove_SNs_cols_from_all_cpo_dfs()
    
#     #****************************************************************************************************
#     if verbose:
#         print('\n\nAfter removing and/or combining reasons:')
#         for i_coll, mecpo_coll_i_w_df_name in enumerate(mecpo_colls_with_cpo_df_names):
#             print(f"MECPO Collection {i_coll}")
#             for an_key in mecpo_coll_i_w_df_name[0].mecpo_an_keys:
#                 print(f"\t\t{an_key}, '{rcpo_dfs_name_outg}' shape: {mecpo_coll_i_w_df_name[0].get_cpo_df(an_key, rcpo_dfs_name_outg).shape}")

In [22]:
outg_mdlr = OutageModeler()

In [23]:
outg_mdlr.set_mecpx_colls(
    mecpx_coll_outg = copy.deepcopy(we_dict['outg']), 
    mecpx_coll_otbl = copy.deepcopy(we_dict['otbl']), 
    mecpx_coll_prbl = copy.deepcopy(we_dict['prbl']), 
)

In [24]:
outg_mdlr.perform_similarity_operations()
outg_mdlr.perform_reduction_operations()
outg_mdlr.compile_merged_dfs()

In OutageModeler.perform_similarity_operations
Starting shapes:
MECPO Collection 0
		01-06 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1722, 43)
		06-11 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1779, 40)
		11-16 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1452, 39)
		16-21 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1527, 42)
		21-26 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1672, 36)
		26-31 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1782, 44)
MECPO Collection 1
		01-06 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2440, 41)
		06-11 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2287, 41)
		11-16 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2726, 41)
		16-21 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2156, 40)
		21-26 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2471, 41)
		26-31 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2760, 37)
MECPO Collection 2
		01-06 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (978, 40)
		06-11 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (955, 41)
		11-16 Days, 'rcpo_df_norm_by_xfmr_nSNs'

In [25]:
if build_mecpo_colls:
    #****************************************************************************************************
    if verbose:
        print('Starting shapes:')
        #-----
        print('\tmecpo_coll_outg')
        for an_key in mecpo_coll_outg.mecpo_an_keys:
            print(f"\t\t{an_key}, '{rcpo_dfs_name_outg}' shape: {mecpo_coll_outg.get_cpo_df(an_key, rcpo_dfs_name_outg).shape}")
        #-----
        print('\tmecpo_coll_otbl')
        for an_key in mecpo_coll_otbl.mecpo_an_keys:
            print(f"\t\t{an_key}, '{rcpo_dfs_name_otbl}' shape: {mecpo_coll_otbl.get_cpo_df(an_key, rcpo_dfs_name_otbl).shape}")
        #-----
        print('\tmecpo_coll_prbl')
        for an_key in mecpo_coll_prbl.mecpo_an_keys:
            print(f"\t\t{an_key}, '{rcpo_dfs_name_prbl}' shape: {mecpo_coll_prbl.get_cpo_df(an_key, rcpo_dfs_name_prbl).shape}")
    #****************************************************************************************************
    
    #----------------------------------------------------------------------------------------------------
    # Similarity operations
    #----------------------------------------------------------------------------------------------------
    # First, make columns equal between MECPOAn objects within each MECPOCollection
    mecpo_coll_outg.make_cpo_columns_equal(drop_empty_cpo_dfs=True)
    mecpo_coll_otbl.make_cpo_columns_equal(drop_empty_cpo_dfs=True)
    mecpo_coll_prbl.make_cpo_columns_equal(drop_empty_cpo_dfs=True)

    #-------------------------
    # Now, make columns equal between the MECPOCollections
    MECPOCollection.make_cpo_columns_equal_between_mecpo_colls(
        mecpo_colls = [
            mecpo_coll_outg, 
            mecpo_coll_otbl, 
            mecpo_coll_prbl
        ], 
        drop_empty_cpo_dfs=True
    )

    #-------------------------
    # If not all same cpo_df names are used between collections, then one should call 
    #   MECPOCollection.make_mixed_cpo_columns_equal_between_mecpo_colls.
    if not(rcpo_dfs_name_outg==rcpo_dfs_name_otbl==rcpo_dfs_name_prbl):
        MECPOCollection.make_mixed_cpo_columns_equal_between_mecpo_colls(
            mecpo_colls_with_cpo_df_names = [
                [mecpo_coll_outg, rcpo_dfs_name_outg], 
                [mecpo_coll_otbl, rcpo_dfs_name_otbl], 
                [mecpo_coll_prbl, rcpo_dfs_name_prbl]
            ], 
            segregate_by_mecpo_an_keys=False
        )
    #-------------------------
    if not(icpo_dfs_name_outg==icpo_dfs_name_otbl==icpo_dfs_name_prbl):
        MECPOCollection.make_mixed_cpo_columns_equal_between_mecpo_colls(
            mecpo_colls_with_cpo_df_names = [
                [mecpo_coll_outg, icpo_dfs_name_outg], 
                [mecpo_coll_otbl, icpo_dfs_name_otbl], 
                [mecpo_coll_prbl, icpo_dfs_name_prbl]
            ], 
            segregate_by_mecpo_an_keys=False
        )
        
    #****************************************************************************************************
    if verbose:
        print('\n\nAfter making columns equal amongst collections:')
        #-----
        print('\tmecpo_coll_outg')
        for an_key in mecpo_coll_outg.mecpo_an_keys:
            print(f"\t\t{an_key}, '{rcpo_dfs_name_outg}' shape: {mecpo_coll_outg.get_cpo_df(an_key, rcpo_dfs_name_outg).shape}")
        #-----
        print('\tmecpo_coll_otbl')
        for an_key in mecpo_coll_otbl.mecpo_an_keys:
            print(f"\t\t{an_key}, '{rcpo_dfs_name_otbl}' shape: {mecpo_coll_otbl.get_cpo_df(an_key, rcpo_dfs_name_otbl).shape}")
        #-----
        print('\tmecpo_coll_prbl')
        for an_key in mecpo_coll_prbl.mecpo_an_keys:
            print(f"\t\t{an_key}, '{rcpo_dfs_name_prbl}' shape: {mecpo_coll_prbl.get_cpo_df(an_key, rcpo_dfs_name_prbl).shape}")
    #****************************************************************************************************
    
    #----------------------------------------------------------------------------------------------------
    # Remove and/or combine reasons
    #----------------------------------------------------------------------------------------------------
    #-------------------------
    # Remove all reasons containing 'cleared'
    mecpo_coll_outg.remove_reasons_from_all_rcpo_dfs(
        regex_patterns_to_remove = regex_to_remove_patterns, 
        ignore_case              = regex_to_remove_ignore_case
    )
    mecpo_coll_otbl.remove_reasons_from_all_rcpo_dfs(
        regex_patterns_to_remove = regex_to_remove_patterns, 
        ignore_case              = regex_to_remove_ignore_case
    )
    mecpo_coll_prbl.remove_reasons_from_all_rcpo_dfs(
        regex_patterns_to_remove = regex_to_remove_patterns, 
        ignore_case              = regex_to_remove_ignore_case
    )

    #-------------------------
    # Combine reasons using the standard combine (see dflt_patterns_and_replace in MECPODf.combine_cpo_df_reasons
    #   for the list of default patterns_and_replace)
    red_to_org_cols_dicts_outg = mecpo_coll_outg.combine_reasons_in_all_rcpo_dfs(
        initial_strip               = True, 
        initial_punctuation_removal = True, 
        return_red_to_org_cols_dict = True
    )
    red_to_org_cols_dicts_otbl = mecpo_coll_otbl.combine_reasons_in_all_rcpo_dfs(
        initial_strip               = True, 
        initial_punctuation_removal = True, 
        return_red_to_org_cols_dict = True
    ) 
    red_to_org_cols_dicts_prbl = mecpo_coll_prbl.combine_reasons_in_all_rcpo_dfs(
        initial_strip               = True, 
        initial_punctuation_removal = True, 
        return_red_to_org_cols_dict = True
    )
    #-------------------------
    # Build power down minus power up counts
    if include_power_down_minus_up:
        mecpo_coll_outg.delta_cpo_df_reasons_in_all_rcpo_dfs(
            reasons_1         = pd_col,
            reasons_2         = pu_col,
            delta_reason_name = pd_m_pu_col
        )
        mecpo_coll_otbl.delta_cpo_df_reasons_in_all_rcpo_dfs(
            reasons_1         = pd_col,
            reasons_2         = pu_col,
            delta_reason_name = pd_m_pu_col
        )
        mecpo_coll_prbl.delta_cpo_df_reasons_in_all_rcpo_dfs(
            reasons_1         = pd_col,
            reasons_2         = pu_col,
            delta_reason_name = pd_m_pu_col
        )
        
    #-------------------------
    # Don't want to include SNs or nSNs cols (and similar) in plotting, so remove
    mecpo_coll_outg.remove_SNs_cols_from_all_cpo_dfs()
    mecpo_coll_otbl.remove_SNs_cols_from_all_cpo_dfs()
    mecpo_coll_prbl.remove_SNs_cols_from_all_cpo_dfs()
    
    #****************************************************************************************************
    if verbose:
        print('\n\nAfter removing and/or combining reasons:')
        #-----
        print('\tmecpo_coll_outg')
        for an_key in mecpo_coll_outg.mecpo_an_keys:
            print(f"\t\t{an_key}, '{rcpo_dfs_name_outg}' shape: {mecpo_coll_outg.get_cpo_df(an_key, rcpo_dfs_name_outg).shape}")
        #-----
        print('\tmecpo_coll_otbl')
        for an_key in mecpo_coll_otbl.mecpo_an_keys:
            print(f"\t\t{an_key}, '{rcpo_dfs_name_otbl}' shape: {mecpo_coll_otbl.get_cpo_df(an_key, rcpo_dfs_name_otbl).shape}")
        #-----
        print('\tmecpo_coll_prbl')
        for an_key in mecpo_coll_prbl.mecpo_an_keys:
            print(f"\t\t{an_key}, '{rcpo_dfs_name_prbl}' shape: {mecpo_coll_prbl.get_cpo_df(an_key, rcpo_dfs_name_prbl).shape}")
    #****************************************************************************************************
    
    #----------------------------------------------------------------------------------------------------
    # Get merged DFs from collections
    #----------------------------------------------------------------------------------------------------
    merged_df_outg = mecpo_coll_outg.get_merged_cpo_dfs(
        cpo_df_name                         = rcpo_dfs_name_outg, 
        cpo_df_subset_by_mjr_mnr_cause_args = None, 
        max_total_counts                    = max_total_counts, 
        how_max_total_counts                = how_max_total_counts
    )
    merged_df_otbl = mecpo_coll_otbl.get_merged_cpo_dfs(
        cpo_df_name                         = rcpo_dfs_name_otbl, 
        cpo_df_subset_by_mjr_mnr_cause_args = None, 
        max_total_counts                    = max_total_counts, 
        how_max_total_counts                = how_max_total_counts
    )
    merged_df_prbl = mecpo_coll_prbl.get_merged_cpo_dfs(
        cpo_df_name                         = rcpo_dfs_name_prbl, 
        cpo_df_subset_by_mjr_mnr_cause_args = None, 
        max_total_counts                    = max_total_counts, 
        how_max_total_counts                = how_max_total_counts
    )

    #-------------------------
    # Make sure all SNs columns are removed
    merged_df_outg = MECPODf.remove_SNs_cols_from_rcpo_df(merged_df_outg)
    merged_df_otbl = MECPODf.remove_SNs_cols_from_rcpo_df(merged_df_otbl)
    merged_df_prbl = MECPODf.remove_SNs_cols_from_rcpo_df(merged_df_prbl)
    
    #-------------------------
    # Sort columns and make sure all columns equal for all
    merged_df_outg = merged_df_outg[merged_df_outg.columns.sort_values()]
    merged_df_otbl = merged_df_otbl[merged_df_otbl.columns.sort_values()]
    merged_df_prbl = merged_df_prbl[merged_df_prbl.columns.sort_values()]
    #-----
    assert(all(merged_df_otbl.columns==merged_df_outg.columns))
    assert(all(merged_df_otbl.columns==merged_df_prbl.columns))
    
    #----------------------------------------------------------------------------------------------------
    # Build counts series
    #----------------------------------------------------------------------------------------------------
    counts_series_outg = mecpo_coll_outg.get_counts_series(rcpo_dfs_name_outg, False)
    counts_series_otbl = mecpo_coll_otbl.get_counts_series(rcpo_dfs_name_otbl, False)
    counts_series_prbl = mecpo_coll_prbl.get_counts_series(rcpo_dfs_name_prbl, False)

Starting shapes:
	mecpo_coll_outg
		01-06 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1722, 43)
		06-11 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1779, 40)
		11-16 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1452, 39)
		16-21 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1527, 42)
		21-26 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1672, 36)
		26-31 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (1782, 44)
	mecpo_coll_otbl
		01-06 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2440, 41)
		06-11 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2287, 41)
		11-16 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2726, 41)
		16-21 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2156, 40)
		21-26 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2471, 41)
		26-31 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (2760, 37)
	mecpo_coll_prbl
		01-06 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (978, 40)
		06-11 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (955, 41)
		11-16 Days, 'rcpo_df_norm_by_xfmr_nSNs' shape: (961, 38)
		16-21 Days, 'rcpo_df_norm_by_xfmr

In [26]:
print(merged_df_outg.equals(outg_mdlr.merged_df_outg))
print(merged_df_otbl.equals(outg_mdlr.merged_df_otbl))
print(merged_df_prbl.equals(outg_mdlr.merged_df_prbl))

True
True
True


In [27]:
print(counts_series_outg.equals(outg_mdlr.counts_series_outg))
print(counts_series_otbl.equals(outg_mdlr.counts_series_otbl))
print(counts_series_prbl.equals(outg_mdlr.counts_series_prbl))

True
True
True


In [28]:
assert(0)

AssertionError: 

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# SAVE OR LOAD
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
if build_mecpo_colls:
    merged_df_outg.to_pickle(os.path.join(save_dir_model_base, 'merged_df_outg.pkl'))
    merged_df_otbl.to_pickle(os.path.join(save_dir_model_base, 'merged_df_otbl.pkl'))
    merged_df_prbl.to_pickle(os.path.join(save_dir_model_base, 'merged_df_prbl.pkl'))
    #-------------------------
    with open(os.path.join(save_dir_model_base, 'mecpo_coll_outg.pkl'), 'wb') as handle:
        pickle.dump(mecpo_coll_outg, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(save_dir_model_base, 'mecpo_coll_otbl.pkl'), 'wb') as handle:
        pickle.dump(mecpo_coll_otbl, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(save_dir_model_base, 'mecpo_coll_prbl.pkl'), 'wb') as handle:
        pickle.dump(mecpo_coll_prbl, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #-------------------------
    with open(os.path.join(save_dir_model_base, 'counts_series_outg.pkl'), 'wb') as handle:
        pickle.dump(counts_series_outg, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(save_dir_model_base, 'counts_series_otbl.pkl'), 'wb') as handle:
        pickle.dump(counts_series_otbl, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(save_dir_model_base, 'counts_series_prbl.pkl'), 'wb') as handle:
        pickle.dump(counts_series_prbl, handle, protocol=pickle.HIGHEST_PROTOCOL)     
    #-------------------------
    CustomWriter.output_dict_to_json(
        os.path.join(save_dir_model_base, 'mecpo_build_info_dict.json'), 
        mecpo_build_info_dict
    )

In [ ]:
if not build_mecpo_colls:
    merged_df_outg=pd.read_pickle(os.path.join(save_dir_model_base, 'merged_df_outg.pkl'))
    merged_df_otbl=pd.read_pickle(os.path.join(save_dir_model_base, 'merged_df_otbl.pkl'))
    merged_df_prbl=pd.read_pickle(os.path.join(save_dir_model_base, 'merged_df_prbl.pkl'))
    #-------------------------
    # with open(os.path.join(save_dir_model_base, 'mecpo_coll_outg.pkl'), 'rb') as handle:
    #     mecpo_coll_outg = pickle.load(handle)
    # with open(os.path.join(save_dir_model_base, 'mecpo_coll_otbl.pkl'), 'rb') as handle:
    #     mecpo_coll_otbl = pickle.load(handle)
    # with open(os.path.join(save_dir_model_base, 'mecpo_coll_prbl.pkl'), 'rb') as handle:
    #     mecpo_coll_prbl = pickle.load(handle)
    #-------------------------
    with open(os.path.join(save_dir_model_base, 'counts_series_outg.pkl'), 'rb') as handle:
        counts_series_outg = pickle.load(handle)
    with open(os.path.join(save_dir_model_base, 'counts_series_otbl.pkl'), 'rb') as handle:
        counts_series_otbl = pickle.load(handle)
    with open(os.path.join(save_dir_model_base, 'counts_series_prbl.pkl'), 'rb') as handle:
        counts_series_prbl = pickle.load(handle)
    #-------------------------
    with open(os.path.join(save_dir_model_base, 'mecpo_build_info_dict.json'), 'rb') as handle:
        mecpo_build_info_dict = json.load(handle)

    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    merged_df_prbl.index.names     = ['trsf_pole_nb', 'no_outg_rec_nb']
    counts_series_prbl.index.names = ['trsf_pole_nb', 'no_outg_rec_nb']

### Initiate summary_dict

In [ ]:
# Building time infos and EEMSP takes a while, so I suggest building these
#   once, saving them, and loading each time
# ==> These should generally be set to False
build_time_infos_dfs = False
build_eemsp          = False

In [ ]:
#-------------------------
random_state = None
#-------------------------
n_top_reasons_to_inclue = 10
# n_top_reasons_to_inclue = None
combine_others          = True
#-------------------------
merge_eemsp = True
mult_strategy='agg'
#-------------------------
include_month = True
#-------------------------
an_keys_to_drop = None
# an_keys_to_drop = ['00-01 Days']
# an_keys_to_drop = ['01-06 Days']
#-------------------------
# date_0_train   = pd.to_datetime('2023-04-01')
# date_1_train   = pd.to_datetime('2023-09-30')
# #-----
# date_0_test    = pd.to_datetime('2023-04-01')
# date_1_test    = pd.to_datetime('2023-09-30')
# #-----
# date_0_holdout = pd.to_datetime('2022-06-01')
# date_1_holdout = pd.to_datetime('2023-09-30')

date_0_train   = pd.to_datetime('2023-04-01')
date_1_train   = pd.to_datetime('2023-11-30')
#-----
date_0_test    = pd.to_datetime('2023-04-01')
date_1_test    = pd.to_datetime('2023-11-30')
#-----
date_0_holdout = pd.to_datetime('2022-06-01')
date_1_holdout = pd.to_datetime('2023-11-30')

#-------------------------
test_size                = 0.33
get_train_test_by_date   = False
split_train_test_by_outg = True 
#-------------------------
create_validation_set = False
val_size              = 0.10 #w.r.t to train size (i.e., w.r.t 1.0-test_size)
#-------------------------
run_scaler=True
#-------------------------
run_PCA = False
pca_n_components=0.95
#-------------------------
remove_others_from_outages=False
#-------------------------
# min_pct_target_1 = 25
min_pct_target_1 = None
#-------------------------
reduce_train_size = False
red_test_size = 0.75 #Amount kept will be 1.0-red_test_size
#-------------------------

In [ ]:
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'LOCATION_ID'), 
#             value=merged_df_outg_w_DOVS.index.get_level_values(1), 
#             comparison_operator='=='
#         ), 
#         dict(
#             column=('outg_dummy_lvl_0', 'EQUIP_TYP_NM'), 
#             value=xfmr_equip_typ_nms_of_interest, 
#             comparison_operator='isin'
#         )
#     ]
# )
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'LOCATION_ID'), 
#             value=merged_df_outg_w_DOVS.index.get_level_values(1), 
#             comparison_operator='=='
#         )
#     ]
# )
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'MNR_CAUSE_NM'), 
#             value='EQUIPMENT FAILURE', 
#             comparison_operator='=='
#         )
#     ]
# )
#-----
slicer = DFSlicer()
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'EQUIP_TYP_NM'), 
#             value='CONDUCTOR OVERHEAD', 
#             comparison_operator='=='
#         )
#     ]
# )
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'LOCATION_ID'), 
#             value=merged_df_outg_w_DOVS.index.get_level_values(1), 
#             comparison_operator='=='
#         ), 
#         dict(
#             column=('outg_dummy_lvl_0', 'OUTG_REC_NB'), 
#             value=outgs_w_single_xfmr, 
#             comparison_operator='isin'
#         )
#     ]
# )
# merged_df_outg_w_DOVS[('outg_dummy_lvl_0', 'OUTG_REC_NB')] = merged_df_outg_w_DOVS.index.get_level_values(0)
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'MJR_CAUSE_NM'), 
#             value='DISTRIBUTION LINE', 
#             comparison_operator='=='
#         ), 
#         dict(
#             column=('outg_dummy_lvl_0', 'MNR_CAUSE_NM'), 
#             value='EQUIPMENT FAILURE', 
#             comparison_operator='=='
#         )
#     ]
# )

In [ ]:
OutageModeler.get_default_mecpx_build_info_dict().keys()

In [ ]:
OutageModeler.get_default_summary_dict().keys()

In [ ]:
assert(set(OutageModeler.get_default_mecpx_build_info_dict().keys()).intersection(set(OutageModeler.get_default_summary_dict().keys()))==set())
acceptable_kwargs = list(OutageModeler.get_default_mecpx_build_info_dict().keys()) + list(OutageModeler.get_default_summary_dict().keys())

In [ ]:
#----------------------------------------------------------------------------------------------------
an_keys = natsorted(merged_df_outg.columns.get_level_values(0).unique().tolist())
assert(an_keys==natsorted(merged_df_otbl.columns.get_level_values(0).unique().tolist()))
assert(an_keys==natsorted(merged_df_prbl.columns.get_level_values(0).unique().tolist()))
#----------------------------------------------------------------------------------------------------
summary_dict = dict()
#-------------------------
summary_dict['an_keys']                    = an_keys
#-------------------------
summary_dict['random_state']               = random_state
#-------------------------
summary_dict['n_top_reasons_to_inclue']    = n_top_reasons_to_inclue
summary_dict['combine_others']             = combine_others
#-------------------------
summary_dict['merge_eemsp']                = merge_eemsp
summary_dict['eemsp_mult_strategy']        = mult_strategy
#-------------------------
summary_dict['include_month']              = include_month
#-------------------------
summary_dict['an_keys_to_drop']            = an_keys_to_drop
#-------------------------
# NOTE: Timestamp is not JSON serializable, hence the need for strftime below
summary_dict['date_0_train']               = date_0_train.strftime('%Y-%m-%d %H:%M:%S')
summary_dict['date_1_train']               = date_1_train.strftime('%Y-%m-%d %H:%M:%S')
#-----
summary_dict['date_0_test']                = date_0_test.strftime('%Y-%m-%d %H:%M:%S')
summary_dict['date_1_test']                = date_1_test.strftime('%Y-%m-%d %H:%M:%S')
#-----
summary_dict['date_0_holdout']             = date_0_holdout.strftime('%Y-%m-%d %H:%M:%S')
summary_dict['date_1_holdout']             = date_1_holdout.strftime('%Y-%m-%d %H:%M:%S')
#-------------------------
summary_dict['test_size']                  = test_size
summary_dict['get_train_test_by_date']     = get_train_test_by_date
summary_dict['split_train_test_by_outg']   = split_train_test_by_outg
#-------------------------
summary_dict['create_validation_set']      = create_validation_set
summary_dict['val_size']                   = val_size
#-------------------------
summary_dict['run_scaler']                 = run_scaler
#-------------------------
summary_dict['run_PCA']                    = run_PCA
summary_dict['pca_n_components']           = pca_n_components
#-------------------------
summary_dict['slicer']                     = slicer.as_dict()
#-------------------------
summary_dict['remove_others_from_outages'] = remove_others_from_outages
#-------------------------
summary_dict['min_pct_target_1']           = min_pct_target_1
#-------------------------
summary_dict['reduce_train_size']          = reduce_train_size
summary_dict['red_test_size']              = red_test_size

### !!!!!!! No outage data have indices backwards from outages!!!!!

In [ ]:
merged_df_otbl = merged_df_otbl.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb'])
counts_series_otbl = counts_series_otbl.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb']).squeeze()
#-----
merged_df_prbl = merged_df_prbl.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb'])
counts_series_prbl = counts_series_prbl.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb']).squeeze()

In [ ]:
merged_df_outg.columns.get_level_values(1).unique().tolist()

In [ ]:
if Path(save_dir_model_base).name=='Models' or Path(save_dir_model_base).name=='Models_00_05':
    is_norm=True
elif Path(save_dir_model_base).name=='Models_raw' or Path(save_dir_model_base).name=='Models_00_05_raw':
    is_norm=False
else:
    assert(0)

if n_top_reasons_to_inclue is not None:
    # NOTE: Cannot do get_top_reasons_subset_from_merged_cpo_df for each, as they will then in general have unequal columns!
    merged_df_outg, [merged_df_otbl, merged_df_prbl] = MECPOCollection.get_top_reasons_subset_from_merged_cpo_df_and_project_from_others(
        merged_cpo_df             = merged_df_outg,
        other_dfs_w_counts_series = [ 
            [merged_df_otbl, counts_series_otbl], 
            [merged_df_prbl, counts_series_prbl]
        ], 
        how                       = 'per_mecpo_an', 
        n_reasons_to_include      = n_top_reasons_to_inclue,
        combine_others            = combine_others,
        output_combine_others_col = 'Other Reasons',
        SNs_tags                  = None, 
        is_norm                   = is_norm, 
        counts_series             = counts_series_outg
    )

In [ ]:
print(merged_df_outg.shape)
print(merged_df_otbl.shape)
print(merged_df_prbl.shape)

In [ ]:
print(merged_df_outg.shape)
print(merged_df_otbl.shape)
print(merged_df_prbl.shape)

In [ ]:
merged_df_outg=MECPOCollection.get_total_event_counts_for_merged_cpo_df(merged_df_outg)
merged_df_otbl=MECPOCollection.get_total_event_counts_for_merged_cpo_df(merged_df_otbl)
merged_df_prbl=MECPOCollection.get_total_event_counts_for_merged_cpo_df(merged_df_prbl)

In [ ]:
print(merged_df_outg.shape)
print(merged_df_otbl.shape)
print(merged_df_prbl.shape)

In [ ]:
assert(len(set(merged_df_outg.index).difference(set(counts_series_outg.index)))==0)
assert(len(set(merged_df_otbl.index).difference(set(counts_series_otbl.index)))==0)
assert(len(set(merged_df_prbl.index).difference(set(counts_series_prbl.index)))==0)

merged_df_outg = pd.merge(
    merged_df_outg, 
    counts_series_outg.to_frame(name=('nSNs', 'nSNs')), 
    left_index=True, right_index=True, how='inner')

merged_df_otbl = pd.merge(
    merged_df_otbl, 
    counts_series_otbl.to_frame(name=('nSNs', 'nSNs')), 
    left_index=True, right_index=True, how='inner')

merged_df_prbl = pd.merge(
    merged_df_prbl, 
    counts_series_prbl.to_frame(name=('nSNs', 'nSNs')), 
    left_index=True, right_index=True, how='inner')

In [ ]:
print(merged_df_outg.shape)
print(merged_df_otbl.shape)
print(merged_df_prbl.shape)

In [ ]:
merged_df_outg

In [ ]:
print(merged_df_outg.shape)
print(merged_df_outg.index.get_level_values(0).nunique())
print(merged_df_outg.index.get_level_values(1).nunique())

In [ ]:
merged_df_outg.index.get_level_values(1).value_counts()

In [ ]:
# merged_df_outg['01-05 Days']
# merged_df_outg['00-01 Days']

In [ ]:
merged_df_outg

# =========================================================
# =========================================================

# Build/grab time_info DFs

In [ ]:
if build_time_infos_dfs:
    time_infos_df_outg = build_outg_time_infos_df(
        rcpx_df=merged_df_outg.copy(), 
        outg_rec_nb_idfr=('index', 'outg_rec_nb'), 
        dummy_col_levels_prefix='dummy_lvl_',     
    )
    time_infos_df_outg.to_pickle(os.path.join(save_dir_model_base, 'time_infos_df_outg.pkl'))    
    #-------------------------
    ede_data_dirs_otbl       = mecpo_build_info_dict['ede_data_dirs_otbl'] 
    ede_data_dirs_prbl = mecpo_build_info_dict['ede_data_dirs_prbl'] 
    #-----
    no_outg_time_infos_df = build_baseline_time_infos_df(
        ede_data_dirs_bsln = ede_data_dirs_otbl,
        save_path          = os.path.join(save_dir_model_base, 'no_outg_time_infos_df.pkl')
    )
    #-----
    no_outg_time_infos_prstn_df = build_baseline_time_infos_df(
        ede_data_dirs_bsln = ede_data_dirs_prbl,
        save_path          = os.path.join(save_dir_model_base, 'no_outg_time_infos_prstn_df.pkl')
    )
else:
    time_infos_df_outg          = pd.read_pickle(os.path.join(save_dir_model_base, 'time_infos_df_outg.pkl'))
    no_outg_time_infos_df       = pd.read_pickle(os.path.join(save_dir_model_base, 'no_outg_time_infos_df.pkl'))
    no_outg_time_infos_prstn_df = pd.read_pickle(os.path.join(save_dir_model_base, 'no_outg_time_infos_prstn_df.pkl'))
#-------------------------
if 'is_first_after_outg' in no_outg_time_infos_df.index.names:
    no_outg_time_infos_df = no_outg_time_infos_df.droplevel(level='is_first_after_outg', axis=0)
#-----
if 'is_first_after_outg' in no_outg_time_infos_prstn_df.index.names:
    no_outg_time_infos_prstn_df = no_outg_time_infos_prstn_df.droplevel(level='is_first_after_outg', axis=0)
    
#-------------------------
# NOTE: After new DFs are built and saved (using build_baseline_time_infos_df), the if statements
#         below will no longer be necessary
# Typically, want index as ['no_outg_rec_nb', 'trsf_pole_nb']
if no_outg_time_infos_df.index.names!=['no_outg_rec_nb', 'trsf_pole_nb']:
    no_outg_time_infos_df = no_outg_time_infos_df.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb'])
#-----
if no_outg_time_infos_prstn_df.index.names!=['no_outg_rec_nb', 'trsf_pole_nb']:
    no_outg_time_infos_prstn_df = no_outg_time_infos_prstn_df.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb'])
    
#-------------------------
# Make sure time info found for all
# For the baseline data, typically the time dfs will have more entries than the data
#   This can result from, e.g., transformers not registering any events, in which case
#     they obviously will not be found in the data, but will be found in the time dfs because
#     those are built through the collection of run SQL queries
assert(len(set(merged_df_outg.index).difference(set(time_infos_df_outg.index)))==0)
assert(len(set(merged_df_otbl.index).difference(set(no_outg_time_infos_df.index)))==0)
assert(len(set(merged_df_prbl.index).difference(set(no_outg_time_infos_prstn_df.index)))==0)

# =========================================================
# =========================================================

# EEMSP

In [ ]:
if merge_eemsp:
    #----------------------------------------------------------------------------------------------------
    # Build time_infos_df
    #--------------------------------------------------
    time_infos_df_otbl = build_baseline_time_infos_df_for_eemsp(
        bsln_time_infos_df = no_outg_time_infos_df
    )
    #-----
    time_infos_df_prbl = build_baseline_time_infos_df_for_eemsp(
        bsln_time_infos_df  no_outg_time_infos_prstn_df
    )
    #-------------------------
    time_infos_df = build_time_infos_df_for_eemsp(
        time_infos_df_outg=time_infos_df_outg, 
        time_infos_df_otbl=time_infos_df_otbl, 
        time_infos_df_prbl=time_infos_df_prbl
    )
    #-------------------------
    assert(time_infos_df.shape[0]==time_infos_df.reset_index().drop_duplicates().shape[0])
    #-----
    del time_infos_df_outg
    del time_infos_df_otbl
    del time_infos_df_prbl
    
    #----------------------------------------------------------------------------------------------------
    # Grab or build df_eemsp_reduce2
    #--------------------------------------------------
    trsf_pole_nbs = list(set(
        merged_df_outg.index.get_level_values(1).unique().tolist()+
        merged_df_otbl.index.get_level_values(1).unique().tolist()+
        merged_df_prbl.index.get_level_values(1).unique().tolist()
    ))
    #-------------------------
    if build_eemsp:
        df_eemsp_OG, df_eemsp_reduce1, df_eemsp_reduce2 = OutageModeler.build_eemsp_df_for_xfmrs_w_time_info(
            trsf_pole_nbs               = trsf_pole_nbs, 
            time_infos_df               = time_infos_df, 
            mult_strategy               = mult_strategy, 
            batch_size                  = 1000, 
            verbose                     = True, 
            n_update                    = 10, 
            addtnl_kwargs               = None, 
            time_infos_df_info_dict     = None, 
            eemsp_df_info_dict          = None, 
            min_pct_found_in_time_infos = 0.75, 
            save_dfs                    = True, 
            save_dir                    = save_dir_model_base, 
            return_3_dfs                = True
        )
    else:
        df_eemsp_OG      = pd.read_pickle(os.path.join(save_dir_model_base, 'df_eemsp_OG.pkl'))
        df_eemsp_reduce1 = pd.read_pickle(os.path.join(save_dir_model_base, 'df_eemsp_reduce1.pkl'))
        df_eemsp_reduce2 = pd.read_pickle(os.path.join(save_dir_model_base, f'df_eemsp_reduce2_{mult_strategy}.pkl'))

    #----------------------------------------------------------------------------------------------------
    print(f"df_eemsp_reduce2['LOCATION_NB'].nunique() = {df_eemsp_reduce2['location_nb'].nunique()}")
    print(f"len(trsf_pole_nbs)                        = {len(trsf_pole_nbs)}")
    print(f"Diff                                      = {len(trsf_pole_nbs)-df_eemsp_reduce2['location_nb'].nunique()}")
    print()
    #-------------------------
    print("\nShapes BEFORE merging")
    print(f"merged_df_outg.shape          = {merged_df_outg.shape}")
    print(f"merged_df_otbl.shape       = {merged_df_otbl.shape}")
    print(f"merged_df_prbl.shape = {merged_df_prbl.shape}")
    #----------------------------------------------------------------------------------------------------
    merged_df_outg = merge_rcpx_with_eemsp(
        df_rcpx=merged_df_outg, 
        df_eemsp=df_eemsp_reduce2, 
        merge_on_rcpx=['index_0', 'index_1'], 
        merge_on_eems=['outg_rec_nb_to_merge', 'location_nb'], 
        set_index=True
    )
    #-------------------------
    merged_df_otbl = merge_rcpx_with_eemsp(
        df_rcpx=merged_df_otbl, 
        df_eemsp=df_eemsp_reduce2, 
        merge_on_rcpx=['index_0', 'index_1'], 
        merge_on_eems=['outg_rec_nb_to_merge', 'location_nb'], 
        set_index=True
    )
    #-------------------------
    merged_df_prbl = merge_rcpx_with_eemsp(
        df_rcpx=merged_df_prbl, 
        df_eemsp=df_eemsp_reduce2, 
        merge_on_rcpx=['index_0', 'index_1'], 
        merge_on_eems=['outg_rec_nb_to_merge', 'location_nb'], 
        set_index=True
    )
    #-------------------------
    print("\nShapes AFTER merging")
    print(f"merged_df_outg.shape = {merged_df_outg.shape}")
    print(f"merged_df_otbl.shape = {merged_df_otbl.shape}")
    print(f"merged_df_prbl.shape = {merged_df_prbl.shape}")

In [ ]:
if merge_eemsp:
    fig,ax = Plot_General.default_subplots()
    Plot_Hist.plot_hist(
        ax=ax, 
        df=df_eemsp_reduce1.groupby('location_nb').size().to_frame(), 
        x_col=0, 
        min_max_and_bin_size=(0,10,1), 
        plot_sns=True, 
        hist_plot_kwargs=dict(discrete=True)
    )
    ax.set_title('# Xfmrs at Pole Location (Only those with >1)')

In [ ]:
if merge_eemsp:
    fig,ax = Plot_General.default_subplots()
    Plot_Hist.plot_hist(
        ax=ax, 
        df=df_eemsp_reduce1.groupby(['location_nb', 'outg_rec_nb_to_merge']).size().to_frame(), 
        x_col=0, 
        min_max_and_bin_size=(0,10,1), 
        plot_sns=True, 
        hist_plot_kwargs=dict(discrete=True)
    )
    ax.set_title('# Xfmrs at Pole Location (Only those with >1)')

In [ ]:
if merge_eemsp:
    del df_eemsp_OG
    del df_eemsp_reduce1
    del df_eemsp_reduce2

In [ ]:
merged_df_otbl.index.names       = ['outg_rec_nb', 'trsf_pole_nb']
merged_df_prbl.index.names = ['outg_rec_nb', 'trsf_pole_nb']

In [ ]:
if include_month or merge_eemsp:
    # If month or EEMSP is included, we will need the time_info_df data
    #-------------------------
    merged_df_outg = OutageModeler.add_time_infos_features(
        cpx_df              = merged_df_outg, 
        time_infos_df       = time_infos_df, 
        include_eemsp       = merge_eemsp, 
        include_month       = include_month, 
        t_min_col           = 't_min', 
        trsf_install_dt_col = ('EEMSP_0', 'install_dt'), 
        return_trsf_age_col = None, 
        return_month_col    = None, 
        keep_time_infos     = False
    )
    #-----
    merged_df_otbl = OutageModeler.add_time_infos_features(
        cpx_df              = merged_df_otbl, 
        time_infos_df       = time_infos_df, 
        include_eemsp       = merge_eemsp, 
        include_month       = include_month, 
        t_min_col           = 't_min', 
        trsf_install_dt_col = ('EEMSP_0', 'install_dt'), 
        return_trsf_age_col = None, 
        return_month_col    = None, 
        keep_time_infos     = False
    )
    #-----
    merged_df_prbl = OutageModeler.add_time_infos_features(
        cpx_df              = merged_df_prbl, 
        time_infos_df       = time_infos_df, 
        include_eemsp       = merge_eemsp, 
        include_month       = include_month, 
        t_min_col           = 't_min', 
        trsf_install_dt_col = ('EEMSP_0', 'install_dt'), 
        return_trsf_age_col = None, 
        return_month_col    = None, 
        keep_time_infos     = False
    )

In [ ]:
# REMOVING SCHEDULED OUTAGES

print(f'merged_df_outg.shape = {merged_df_outg.shape}')

merged_df_outg =  MECPODf.get_cpo_df_subset_excluding_mjr_mnr_causes( 
    cpo_df=merged_df_outg, 
    mjr_mnr_causes_to_exclude=None, 
    mjr_causes_to_exclude=None,
    mnr_causes_to_exclude=['SCO', 'SO'], 
    outg_rec_nb_col='index'
)

print(f'merged_df_outg.shape = {merged_df_outg.shape}')

In [ ]:
if ('is_outg', 'is_outg') in merged_df_outg.columns:
    merged_df_outg = Utilities_df.move_cols_to_back(merged_df_outg, [('is_outg', 'is_outg')])
if ('is_outg', 'is_outg') in merged_df_otbl.columns:
    merged_df_otbl = Utilities_df.move_cols_to_back(merged_df_otbl, [('is_outg', 'is_outg')])
if ('is_outg', 'is_outg') in merged_df_prbl.columns:
    merged_df_prbl = Utilities_df.move_cols_to_back(merged_df_prbl, [('is_outg', 'is_outg')])

In [ ]:
merged_df_outg

# !~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~

In [ ]:
# Remove NETWORK and PRIMARY trsf_pole_nbs
merged_df_outg = merged_df_outg.loc[~merged_df_outg.index.get_level_values(1).isin(['NETWORK', 'PRIMARY'])]
merged_df_otbl = merged_df_otbl.loc[~merged_df_otbl.index.get_level_values(1).isin(['NETWORK', 'PRIMARY'])]
merged_df_prbl = merged_df_prbl.loc[~merged_df_prbl.index.get_level_values(1).isin(['NETWORK', 'PRIMARY'])]

In [ ]:
# Get DOVS info to be used for setting target values
merged_df_outg_w_DOVS = DOVSOutages.append_outg_info_to_df(
    df=merged_df_outg.copy(), 
    outg_rec_nb_idfr=('index', 'outg_rec_nb'), 
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
)
merged_df_outg_w_DOVS=merged_df_outg_w_DOVS[['outg_dummy_lvl_0']]
merged_df_outg_w_DOVS[('is_outg', 'is_outg')]=0

In [ ]:
# Add 'from_outg' information so I can track how many are in target=1 and target=0 
merged_df_outg[('from_outg', 'from_outg')]          = 1
merged_df_otbl[('from_outg', 'from_outg')]       = 0
merged_df_prbl[('from_outg', 'from_outg')] = 0

# Drop any time periods

In [ ]:
if an_keys_to_drop is not None:
    merged_df_outg = merged_df_outg.drop(columns=an_keys_to_drop, level=0)
    merged_df_otbl = merged_df_otbl.drop(columns=an_keys_to_drop, level=0)
    merged_df_prbl = merged_df_prbl.drop(columns=an_keys_to_drop, level=0)

# BEG DEV

In [ ]:
split_data_dict_outg = OutageModeler.train_test_split_outg(
    df                       = merged_df_outg, 
    split_train_test_by_outg = split_train_test_by_outg, 
    test_size                = test_size, 
    random_state             = random_state, 
    get_train_test_by_date   = get_train_test_by_date,
    date_range_train         = [date_0_train, date_1_train], 
    date_range_test          = [date_0_test,  date_1_test], 
    outg_rec_nb_idfr         = ('index', 'outg_rec_nb'), 
    date_range_holdout       = None, 
    verbose                  = True
)

In [ ]:
split_data_dict_otbl = OutageModeler.train_test_split_bsln(
    df                       = merged_df_otbl, 
    split_train_test_by_outg = split_train_test_by_outg, 
    test_size                = test_size, 
    random_state             = random_state, 
    get_train_test_by_date   = get_train_test_by_date,
    bsln_time_infos_df       = time_infos_df, 
    date_range_train         = [date_0_train, date_1_train], 
    date_range_test          = [date_0_test,  date_1_test], 
    # no_outg_rec_nb_idfr      = ('index', 'no_outg_rec_nb'), 
    no_outg_rec_nb_idfr      = ('index', 'outg_rec_nb'), 
    bsln_time_infos_time_col = 't_min', 
    date_range_holdout       = None, 
    verbose                  = True
)

In [ ]:
split_data_dict_prbl = OutageModeler.train_test_split_bsln(
    df                       = merged_df_prbl, 
    split_train_test_by_outg = split_train_test_by_outg, 
    test_size                = test_size, 
    random_state             = random_state, 
    get_train_test_by_date   = get_train_test_by_date,
    bsln_time_infos_df       = time_infos_df, 
    date_range_train         = [date_0_train, date_1_train], 
    date_range_test          = [date_0_test,  date_1_test], 
    # no_outg_rec_nb_idfr      = ('index', 'no_outg_rec_nb'), 
    no_outg_rec_nb_idfr      = ('index', 'outg_rec_nb'), 
    bsln_time_infos_time_col = 't_min', 
    date_range_holdout       = None, 
    verbose                  = True
)

# END DEV

In [ ]:
addtnl_bsln_frac = 1.0

addtnl_bsln_train   = pd.concat([split_data_dict_otbl['train'],   split_data_dict_prbl['train']])
addtnl_bsln_test    = pd.concat([split_data_dict_otbl['test'],    split_data_dict_prbl['test']])
addtnl_bsln_holdout = pd.concat([split_data_dict_otbl['holdout'], split_data_dict_prbl['holdout']])

assert(0 <= addtnl_bsln_frac <= 1.0)
if addtnl_bsln_frac < 1.0:
    addtnl_bsln_train = addtnl_bsln_train.sample(frac = addtnl_bsln_frac)
    addtnl_bsln_test = addtnl_bsln_test.sample(frac = addtnl_bsln_frac)
    addtnl_bsln_holdout = addtnl_bsln_holdout.sample(frac = addtnl_bsln_frac)

In [ ]:
full_data_df_train   = pd.concat([split_data_dict_outg['train'],   addtnl_bsln_train])
full_data_df_test    = pd.concat([split_data_dict_outg['test'],    addtnl_bsln_test])
full_data_df_holdout = pd.concat([split_data_dict_outg['holdout'], addtnl_bsln_holdout])

#Shuffle the data
full_data_df_train   = full_data_df_train.sample(frac=1)
full_data_df_test    = full_data_df_test.sample(frac=1)
full_data_df_holdout = full_data_df_holdout.sample(frac=1)

full_data_df = pd.concat([full_data_df_train, full_data_df_test, full_data_df_holdout])

# =========================================================
# =========================================================

In [ ]:
full_data_df_test

In [ ]:
if merge_eemsp:
    [
        full_data_df, 
        full_data_df_train, 
        full_data_df_test, 
        full_data_df_holdout
    ], eemsp_enc = OutageModeler.encode_eemsp(
        df             = [
            full_data_df, 
            full_data_df_train, 
            full_data_df_test, 
            full_data_df_holdout
        ], 
        eemsp_enc      = None, 
        eemsp_lvl_0_nm = 'EEMSP_0', 
        cols_to_encode = None, 
        numeric_cols   = ['kva_size', 'install_dt'], 
        run_transform  = True, 
        no_xfrm_df0    = True, 
    )
    #-------------------------
    if save_model:
        joblib.dump(eemsp_enc, os.path.join(save_dir_model, 'eemsp_encoder.joblib'))

# Set target values

In [ ]:
merged_df_outg_w_DOVS_i = slicer.set_simple_column_value(df=merged_df_outg_w_DOVS.copy(), column=('is_outg', 'is_outg'), value=1)
full_outg_idxs_i = merged_df_outg_w_DOVS_i[merged_df_outg_w_DOVS_i[('is_outg', 'is_outg')]==1].index

#NOTE: To achieve exclusion equal to that in set_target_val_1_by_idx, one could use:
#        full_outg_idxs_exclude_i = merged_df_outg_w_DOVS_i[merged_df_outg_w_DOVS_i[('is_outg', 'is_outg')]==0].index
#      then use .drop(index=list(set(full_data_df_i.index).intersection(set(full_outg_idxs_exclude_i))))
#      However, as noted in the function, the methods in set_target_val_1_by_idx are probably safer/more robust

In [ ]:
#-------------------------
[
    full_data_df_i, 
    full_data_df_train_i, 
    full_data_df_test_i, 
    full_data_df_holdout_i
] = OutageModeler.set_target_val_1_by_idx(
    df                         = [
        full_data_df.copy(), 
        full_data_df_train.copy(), 
        full_data_df_test.copy(), 
        full_data_df_holdout.copy()
    ],
    val_1_idxs                 = full_outg_idxs_i,
    remove_others_from_outages = remove_others_from_outages, 
    target_col                 = ('is_outg', 'is_outg'), 
    from_outg_col              = ('from_outg', 'from_outg')
)

In [ ]:
OutageModeler.print_data_composition(
    df            = [full_data_df_train_i, full_data_df_test_i, full_data_df_holdout_i], 
    headline_str  = ['TRAIN',              'TEST',              'HOLDOUT']
)

In [ ]:
if full_data_df_train_i[('is_outg', 'is_outg')].sum()==0:
    print(f"Not enough target value==1 in train\n#target==1 train:   {full_data_df_train_i[('is_outg', 'is_outg')].sum()}")
#-----
if full_data_df_test_i[('is_outg', 'is_outg')].sum()==0:
    print(f"Not enough target value==1 in test\n#target==1 test:    {full_data_df_test_i[('is_outg', 'is_outg')].sum()}")
#-----
if full_data_df_holdout_i.shape[0]>0 and full_data_df_holdout_i[('is_outg', 'is_outg')].sum()==0:
    print(f"Not enough target value==1 in holdout\n#target==1 holdout: {full_data_df_holdout_i[('is_outg', 'is_outg')].sum()}")

In [ ]:
if min_pct_target_1 is not None:
    full_data_df_train_i = ensure_target_val_1_min_pct(
        df=full_data_df_train_i,
        min_pct=min_pct_target_1,
        target_col=('is_outg', 'is_outg'), 
        random_state=random_state, 
        return_discarded=False
    )
    #-----
    full_data_df_test_i = ensure_target_val_1_min_pct(
        df=full_data_df_test_i,
        min_pct=min_pct_target_1,
        target_col=('is_outg', 'is_outg'), 
        random_state=random_state, 
        return_discarded=False
    )
    #-----
    full_data_df_holdout_i = ensure_target_val_1_min_pct(
        df=full_data_df_holdout_i,
        min_pct=min_pct_target_1,
        target_col=('is_outg', 'is_outg'), 
        random_state=random_state, 
        return_discarded=False
    )
    #-----

In [ ]:
OutageModeler.print_data_composition(
    df            = [full_data_df_train_i, full_data_df_test_i, full_data_df_holdout_i], 
    headline_str  = ['TRAIN',              'TEST',              'HOLDOUT']
)

In [ ]:
if ('from_outg', 'from_outg') in full_data_df_i.columns.tolist():
    full_data_df_i = full_data_df_i.drop(columns=[('from_outg', 'from_outg')])
if ('from_outg', 'from_outg') in full_data_df_train_i.columns.tolist():
    full_data_df_train_i = full_data_df_train_i.drop(columns=[('from_outg', 'from_outg')])
if ('from_outg', 'from_outg') in full_data_df_test_i.columns.tolist():
    full_data_df_test_i = full_data_df_test_i.drop(columns=[('from_outg', 'from_outg')])
if ('from_outg', 'from_outg') in full_data_df_holdout_i.columns.tolist():
    full_data_df_holdout_i = full_data_df_holdout_i.drop(columns=[('from_outg', 'from_outg')])

In [ ]:
if reduce_train_size:
    if split_train_test_by_outg:
        full_data_df_train_i, _ = train_test_split_df_by_outage(
            df=full_data_df_train_i, 
            outg_rec_nb_idfr='index_0', 
            test_size=red_test_size, 
            random_state=random_state
        )
    else:
        full_data_df_train_i, _ = train_test_split(
            full_data_df_train_i, 
            test_size=red_test_size, 
            random_state=random_state
        )

# Prepare Data

In [ ]:
#-------------------------
X_train_OG = full_data_df_train_i.iloc[:, :-1].copy()
y_train    = full_data_df_train_i.iloc[:, -1].copy()

X_test_OG = full_data_df_test_i.iloc[:, :-1].copy()
y_test    = full_data_df_test_i.iloc[:, -1].copy()

if full_data_df_holdout_i.shape[0]>0:
    X_holdout = full_data_df_holdout_i.iloc[:, :-1]
    y_holdout = full_data_df_holdout_i.iloc[:, -1]
else:
    X_holdout = pd.DataFrame()
    y_holdout = pd.DataFrame()
#-------------------------
if save_model:
    X_train_OG.head().to_pickle(os.path.join(save_dir_model, 'data_structure_df.pkl'))

In [ ]:
fig, axs = OutageModeler.plot_target_counts_full_train_test(
    df_full     = full_data_df_i, 
    df_train    = y_train, 
    df_test     = y_test, 
    df_holdout  = y_holdout, 
    is_outg_col = ('is_outg', 'is_outg')
)

In [ ]:
create_validation_set

In [ ]:
run_PCA

In [ ]:
run_scaler

In [ ]:
X_holdout

In [ ]:
#-------------------------
if create_validation_set:
    if split_train_test_by_outg:
        X_train_OG, X_val_OG, y_train, y_val = train_test_split_df_group(
            X=X_train_OG, 
            y=y_train, 
            groups=X_train_OG.index.get_level_values(0), 
            test_size=val_size, 
            random_state=random_state
        )
    else:
        X_train_OG, X_val_OG, y_train, y_val = train_test_split(X_train_OG, y_train, test_size=val_size, random_state=random_state)
#-------------------------
if run_scaler:
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_OG)
    #-----
    if create_validation_set:
        X_val   = scaler.transform(X_val_OG)
    X_test  = scaler.transform(X_test_OG)
    if full_data_df_holdout_i.shape[0]>0:
        X_holdout   = scaler.transform(X_holdout)
    #-----
    if save_model:
        joblib.dump(scaler, os.path.join(save_dir_model, 'scaler.joblib'))
else:
    X_train = X_train_OG
    if create_validation_set:
        X_val   = X_val_OG
    X_test  = X_test_OG

In [ ]:
X_train_OG

In [ ]:
X_train.shape

In [ ]:
X_train

In [ ]:
if run_PCA:
    # First, generate a PCA plot, showing the explained variance on the y-axis and number of components on x
    # This is simply to check that the number of components kept looks correct
    pca = PCA()
    pca.fit(X_train)
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    plt.plot(cumsum)
    
    # Now, run the PCA with pca_n_components and perform transforms
    pca=PCA(n_components=pca_n_components)
    X_train = pca.fit_transform(X_train)
    print(f'PCA n-components       = {pca.n_components_}')
    print(f'PCA explained variance = {pca.explained_variance_ratio_.sum()}')
    #-----
    if create_validation_set:
        X_val      = pca.transform(X_val)
    X_test     = pca.transform(X_test)
    if full_data_df_holdout_i.shape[0]>0:
        X_holdout  = pca.transform(X_holdout)

# Dumb Classifier

In [ ]:
dumb_clf = DumbClassifier()
cross_val_score(dumb_clf, X_train, y_train, cv=3, scoring='accuracy')

In [ ]:
# y_pred = dumb_clf.predict(X_test)
# print("ACCURACY  OF THE MODEL: ", accuracy_score(y_test, y_pred))
# print("PRECISION OF THE MODEL: ", precision_score(y_test, y_pred))
# print("RECALL    OF THE MODEL: ", recall_score(y_test, y_pred))

In [ ]:
# confusion_matrix(y_test, y_pred)

# Random Forest

In [ ]:
# forest_clf = RandomForestClassifier(n_estimators = 100)
# forest_clf = RandomForestClassifier(n_estimators = 1000, n_jobs=-1)
# forest_clf = RandomForestClassifier(n_estimators = 1000, criterion='entropy', n_jobs=-1)

# forest_clf = RandomForestClassifier(n_estimators = 1000, n_jobs=-1)
# forest_clf = RandomForestClassifier(n_estimators = 1000, max_depth=25, n_jobs=-1)
forest_clf = RandomForestClassifier(n_estimators = 1000, max_depth=25, n_jobs=None)
# forest_clf = RandomForestClassifier(n_estimators = 1000, max_depth=25, n_jobs=None, class_weight='balanced')
# forest_clf = RandomForestClassifier(n_estimators = 1000, max_depth=None, n_jobs=None) #probably dont want max_depth=None
# forest_clf = RandomForestClassifier(n_estimators = 5000, max_depth=50, n_jobs=None)

In [ ]:
start = time.time()
forest_clf.fit(X_train, y_train)
print(time.time()-start)

In [ ]:
# Combine mecpo_build_info_dict and summary_dict
assert(set(mecpo_build_info_dict.keys()).intersection(set(summary_dict.keys()))==set())
summary_dict = summary_dict | mecpo_build_info_dict
#-------------------------
if save_results:
    CustomWriter.output_dict_to_json(
        os.path.join(save_dir_model, 'summary_dict.json'), 
        summary_dict
    )
#-----
if save_model:
    joblib.dump(forest_clf, os.path.join(save_dir_model, 'forest_clf.joblib'))

In [ ]:
y_pred_train = forest_clf.predict(X_train)
print('*****'*5)
print('TRAINING DATASET')
print('*****'*5)
print(f"#(target==1): {y_train.sum()}")
print(f"#(target==0): {y_train.shape[0]-y_train.sum()}")
print(f"%(target==1): {100*(y_train.sum()/(y_train.shape[0]))}")
print('-----'*5)
print("ACCURACY  OF THE MODEL: ", accuracy_score(y_train, y_pred_train))
print("PRECISION OF THE MODEL: ", precision_score(y_train, y_pred_train))
print("RECALL    OF THE MODEL: ", recall_score(y_train, y_pred_train))
print()

y_pred = forest_clf.predict(X_test)
print('*****'*5)
print('TESTING DATASET')
print('*****'*5)
print(f"#(target==1): {y_test.sum()}")
print(f"#(target==0): {y_test.shape[0]-y_test.sum()}")
print(f"%(target==1): {100*(y_test.sum()/(y_test.shape[0]))}")
print('-----'*5)
print("ACCURACY  OF THE MODEL: ", accuracy_score(y_test, y_pred))
print("PRECISION OF THE MODEL: ", precision_score(y_test, y_pred))
print("RECALL    OF THE MODEL: ", recall_score(y_test, y_pred))
print()

if full_data_df_holdout_i.shape[0]>0:
    y_pred_holdout = forest_clf.predict(X_holdout)
    print('*****'*5)
    print('HOLDOOUT DATASET')
    print('*****'*5)
    print(f"#(target==1): {y_holdout.sum()}")
    print(f"#(target==0): {y_holdout.shape[0]-y_holdout.sum()}")
    print(f"%(target==1): {100*(y_holdout.sum()/(y_holdout.shape[0]))}")
    print('-----'*5)
    print("ACCURACY  OF THE MODEL: ", accuracy_score(y_holdout, y_pred_holdout))
    print("PRECISION OF THE MODEL: ", precision_score(y_holdout, y_pred_holdout))
    print("RECALL    OF THE MODEL: ", recall_score(y_holdout, y_pred_holdout))

In [ ]:
reload(Utilities_df)

In [ ]:
outg_n_xfmrs = get_n_trsf_poles_per_outg(
    df=merged_df_outg, 
    outg_rec_nb_idfr='index_0', 
    trsf_pole_nb_idfr='index_1'
)
outgs_w_single_xfmr = outg_n_xfmrs[outg_n_xfmrs==1].index.tolist()
#-------------------------
n_dir_indir_tp_fn_train = get_n_direct_xfmrs_in_tp_fn(
    data_df=full_data_df_train_i, 
    y_pred=y_pred_train, 
    y_col=('is_outg', 'is_outg'), 
    outgs_w_single_xfmr=outgs_w_single_xfmr, 
    xfmr_equip_typ_nms_of_interest=None, 
    outg_rec_nb_idfr='index_0', 
    trsf_pole_nb_idfr='index_1'
)
#-----
n_dir_indir_tp_fn_test = get_n_direct_xfmrs_in_tp_fn(
    data_df=full_data_df_test_i, 
    y_pred=y_pred, 
    y_col=('is_outg', 'is_outg'), 
    outgs_w_single_xfmr=outgs_w_single_xfmr, 
    xfmr_equip_typ_nms_of_interest=None, 
    outg_rec_nb_idfr='index_0', 
    trsf_pole_nb_idfr='index_1'
)

In [ ]:
reload(Utilities_df)

In [ ]:
idk = pd.DataFrame(columns=['idk'])

In [ ]:
idk.dtypes

In [ ]:
idk

In [ ]:
idk = Utilities_df.convert_col_type(
    df=idk, 
    column='idk', 
    to_type=int, 
    to_numeric_errors='coerce', 
    inplace=False
)

In [ ]:
idk.dtypes

In [ ]:
save_figs    = save_results
fig_save_dir = save_dir_model

In [ ]:
# fig, axs = Plot_General.default_subplots(
#     n_x=1, 
#     n_y=3, 
#     fig_num=fig_num, 
#     unit_figsize_width=6., 
#     unit_figsize_height=4., 
# )
# Plot_General.adjust_subplots_args(fig, dict(hspace=0.4))
# #-----
# cmd = draw_outg_confusion_matrix(
#     y=y_train, 
#     y_pred=y_pred_train, 
#     title='Train', 
#     normalize=None, 
#     scientific=True, 
#     ax=axs[0]
# )
# #-----
# cmd = draw_outg_confusion_matrix(
#     y=y_test, 
#     y_pred=y_pred, 
#     title='Test', 
#     normalize=None, 
#     scientific=True, 
#     ax=axs[1]
# )
# #-----
# cmd = draw_outg_confusion_matrix(
#     y=y_holdout, 
#     y_pred=y_pred_holdout, 
#     title='Holdout', 
#     normalize=None, 
#     scientific=True, 
#     ax=axs[2]
# )
# #-----
# if save_figs:
#     if not os.path.exists(fig_save_dir):
#         os.makedirs(fig_save_dir)
#     Plot_General.save_fig(
#         fig=fig, 
#         save_dir=fig_save_dir, 
#         save_name='ConfusionMatrices.png', 
#         bbox_inches='tight'
#     )
# #-----
# fig_num+=1

In [ ]:
fig, axs = Plot_General.default_subplots(
    n_x=1, 
    n_y=2, 
    fig_num=fig_num, 
    unit_figsize_width=6., 
    unit_figsize_height=4., 
)
Plot_General.adjust_subplots_args(fig, dict(hspace=0.4))
#-----
cmd = draw_outg_confusion_matrix(
    y=y_train, 
    y_pred=y_pred_train, 
    title='Train', 
    normalize=None, 
    scientific=False, 
    ax=axs[0], 
    n_dir_indir_tp_fn=n_dir_indir_tp_fn_train
)
#-----
cmd = draw_outg_confusion_matrix(
    y=y_test, 
    y_pred=y_pred, 
    title='Test', 
    normalize=None, 
    scientific=False, 
    ax=axs[1], 
    n_dir_indir_tp_fn=n_dir_indir_tp_fn_test
)
#-----
if save_figs:
    if not os.path.exists(fig_save_dir):
        os.makedirs(fig_save_dir)
    Plot_General.save_fig(
        fig=fig, 
        save_dir=fig_save_dir, 
        save_name='ConfusionMatrices.png', 
        bbox_inches='tight'
    )
#-----
fig_num+=1